In [6]:
import numpy as np 
import pandas as pd
get_ipython().run_line_magic('config', 'Completer.use_jedi = False')
from services_featuers.get_category_vector_svm__ import get_category_vector
from services_featuers.get_category_vector_nn import get_category_vector_nn
from services_featuers.get_emotion_vector import get_emotion_vector_svm
from services_featuers.get_emotion_vector_NN import get_emotion_vecor_nn
import numpy as np


# In[70]:


def concatentate_tow_numpyarray(a,b):
    k=list()
    for i in range(len(b)):
        k.append(a[i].tolist()+b[i].tolist())
    return np.array(k)
def get_vector_category_full(post):
    return concatentate_tow_numpyarray(get_category_vector(post),get_category_vector_nn(post))


# In[71]:


def get_vector_emotion_full(post):
    return concatentate_tow_numpyarray(get_emotion_vector_svm(post),get_emotion_vecor_nn(post))


# In[76]:


def convert_to_one_vector(nparray):
    l=list()
    for s in nparray:
        l=l+s.tolist()
    return np.array(l)
def get_featuers_for_post(post):
    return concatentate_tow_numpyarray(get_vector_category_full(post),get_vector_emotion_full(post))



SVC(probability=True)


In [7]:
data=pd.read_csv("../data/Data_cleaning_with_content.csv")
reactions=['love','wow','angry','sad','care']

In [8]:
data=data.dropna()
contents=data['contentclean'].values
love=data['love'].values
wow=data['wow'].values
sad=data['sigh'].values
angry=data['grrr'].values
care=data['care'].values
category=data['category'].values
sum_reaction=data['sumreactions'].values
love=love/sum_reaction
wow=wow/sum_reaction
angry=angry/sum_reaction
sad=sad/sum_reaction
care=care/sum_reaction

In [9]:
Emotions=np.array([love]).reshape(-1,1)
Emotions=np.insert(Emotions,1,wow,axis=1)
Emotions=np.insert(Emotions,2,angry,axis=1)
Emotions=np.insert(Emotions,3,sad,axis=1)
Emotions=np.insert(Emotions,4,care,axis=1)

In [10]:
Y_train=Emotions

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers

In [12]:
import time
s=time.time()
X_train=get_featuers_for_post(contents) 
print(time.time()-s)

KeyboardInterrupt: 

In [ ]:
print(X_train.shape)
print(Y_train.shape)

# train nn

In [10]:
model=Sequential()
DROPOUT=0.3
model.add(Dense(units=128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='adam',
                loss='mse',
                 metrics=['accuracy'])
model.summary()

model.fit(X_train, Y_train, epochs=20, batch_size=32,validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               4736      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 21,893
Trainable params: 21,893
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export 

In [11]:
model.save("../Final_models_svm/model_nn.h5")

# train svm

In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [14]:
reg_love = LinearRegression().fit(X_train,love)
reg_wow = LinearRegression().fit(X_train,wow)
reg_angry = LinearRegression().fit(X_train,angry)
reg_sad = LinearRegression().fit(X_train,sad)
reg_care = LinearRegression().fit(X_train,care)

In [15]:
svm_models={"love":reg_love,"wow":reg_wow,"angry":reg_angry,"sad":reg_sad,"care":reg_care}

In [18]:
import pickle
for k,v in svm_models.items():
    
    with open("../Final_models_svm/reg_"+k+".pickle","wb") as file:
        pickle.dump(v,file)